# Examples of solutions to SQL tasks using Python

The Transactions.xlsx file contains the following databases:
- transaction lines (Trans_Line)
- transaction headers (Trans_Header)
- stores (Store)
- inventory (Item)

The "Description" tab contains a description of the tables.

You should prepare SQL queries that will answer the following questions:

a) What share of transactions by sales channel in stream A are transactions containing at least two products? Only sales lines should be considered, excluding returns.

b) Which brand has the highest share of returns in Poland?

c) Which product (product number) achieved the highest % margin in the period July 5-15, 2019 in stream C? Only sales lines should be considered, excluding returns.

d) What margin would the PL country achieve without the price reduction?

e) What is the share of all transactions in July in stream A in individual countries that includes both category B and K inventories (at least 1 sales item each)? Only sales lines should be considered, excluding returns.

In [ ]:
import pandas as pd
import sqlite3

In [2]:
TransLine = pd.read_excel(
    'Transactions.xlsx', 
    sheet_name='Trans_Line',
    header=0).dropna()

In [3]:
TransHeader= pd.read_excel(
    'Transactions.xlsx', 
    sheet_name='Trans_Header',
    header=0).dropna()

In [4]:
Store = pd.read_excel(
    'Transactions.xlsx', 
    sheet_name='Store',
    header=0).dropna()

In [5]:
Item = pd.read_excel(
    'Transactions.xlsx', 
    sheet_name='Item',
    header=0).dropna()

In [6]:
db_conn = sqlite3.connect("TransNew1.db")

In [7]:
c = db_conn.cursor()

In [8]:
c.execute(
    """
    CREATE TABLE Store (
        store_no TEXT NOT NULL,
        Stream TEXT,
        Region INTEGER,
        Channel TEXT,
        Country TEXT,
        PRIMARY KEY(store_no)
        );
     """
)

In [9]:
c.execute(
    """
    CREATE TABLE Item (
        item_no TEXT,
        Category_Code TEXT,
        Brand_Code TEXT,
        PRIMARY KEY(item_no)
        );
     """
)

In [10]:
c.execute(
    """
    CREATE TABLE TransHeader (
        store_no TEXT NOT NULL,
        transaction_no TEXT NOT NULL,
        status TEXT NOT NULL,
        date TEXT NOT NULL
        );
     """
)

In [11]:
c.execute(
    """
    CREATE TABLE TransLine (
        store_no TEXT NOT NULL,
        transaction_no TEXT NOT NULL,
        item_no TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        net_amount REAL NOT NULL,
        cost_amount REAL NOT NULL,
        discount_amount REAL NOT NULL
        );
     """
)

In [12]:
Store.to_sql('Store', db_conn, if_exists='append', index=False)
Item.to_sql('Item', db_conn, if_exists='append', index=False)
TransHeader.to_sql('TransHeader', db_conn, if_exists='append', index=False)
TransLine.to_sql('TransLine', db_conn, if_exists='append', index=False)

158065

# Exercises

a) What share of transactions by sales channel in stream A are transactions containing at least two products? Only sales lines should be considered, excluding returns.

In [82]:
zad_a=("""SELECT Store.Channel,
(SELECT Count(TransLine.transaction_no) as tc 
      FROM TransLine 
      JOIN Store
       ON TransLine.store_no=Store.store_no
       WHERE Store.Stream = 'A'
       AND TransLine.quantity>=2
       GROUP BY Store.Channel) as Over2, 
       Count(TransLine.transaction_no) as TransCountTotal
       FROM TransLine
       JOIN Store
       ON TransLine.store_no=Store.store_no
       WHERE Store.Stream = 'A'
       GROUP BY Store.Channel
""")

In [83]:
pd.read_sql(zad_a,db_conn)

,Channel,Over2,TransCountTotal
0,EC,923,63054


b) Which brand has the highest share of returns in Poland?

In [204]:
zad_b=("""SELECT Item.Brand_Code, sum(TransLine.quantity) as Brand_Returns
       FROM TransLine
       JOIN Store ON TransLine.store_no=Store.store_no
       JOIN Item ON TransLine.item_no=Item.item_no       
       WHERE Store.Country = 'PL'
       AND TransLine.quantity<0
       GROUP BY Item.Brand_Code
       ORDER BY Brand_Returns
""")

In [205]:
pd.read_sql(zad_b,db_conn)

,Brand_Code,Brand_Returns
0,B4,-1924
1,B2,-718
2,B7,-225
3,B1,-111
4,B3,-65
5,B5,-51
6,B6,-50


In [236]:
zad_b=("""SELECT Item.Brand_Code, sum(TransLine.quantity) as Brand_Returns,
        CAST((sum(TransLine.quantity)*100/(SELECT  sum(TransLine.quantity) as Brand_Returns
       FROM TransLine
       JOIN Store ON TransLine.store_no=Store.store_no      
       WHERE Store.Country = 'PL'
       AND TransLine.quantity<0)) as float) as Total_Returns_PL
       FROM TransLine
       JOIN Store ON TransLine.store_no=Store.store_no
       JOIN Item ON TransLine.item_no=Item.item_no       
       WHERE Store.Country = 'PL'
       AND TransLine.quantity<0
       GROUP BY Item.Brand_Code
       ORDER BY Brand_Returns
""")

In [237]:
pd.read_sql(zad_b,db_conn)

,Brand_Code,Brand_Returns,Total_Returns_PL
0,B4,-1924,61.0
1,B2,-718,22.0
2,B7,-225,7.0
3,B1,-111,3.0
4,B3,-65,2.0
5,B5,-51,1.0
6,B6,-50,1.0


c) Which product (product number) achieved the highest % margin in the period July 5-15, 2019 in stream C? Only sales lines should be considered, excluding returns.

In [288]:
zad_c=("""SELECT TransLine.item_no, sum(TransLine.net_amount- TransLine.cost_amount + TransLine.discount_amount) as Margin
       FROM TransLine
       LEFT JOIN Store ON TransLine.store_no=Store.store_no
       LEFT JOIN TransHeader ON TransLine.store_no=TransHeader.store_no  AND TransLine.transaction_no= TransHeader.transaction_no
       WHERE Store.Stream = 'C'
       AND TransHeader.date BETWEEN '2019-07-05' AND '2019-07-16' and TransLine.quantity <>-1
       GROUP BY TransLine.item_no
       ORDER BY Margin DESC LIMIT 1
""")

In [289]:
pd.read_sql(zad_c,db_conn)

,item_no,Margin
0,385181,920.51


d) What margin would the PL country achieve without the price reduction?

In [151]:
zad_d=("""SELECT ROUND(SUM(TransLine.net_amount- TransLine.cost_amount)/SUM(TransLine.net_amount)*100, 2) + ' %' as PL_Margin
       FROM TransLine
       JOIN Store
       ON TransLine.store_no=Store.store_no
       WHERE Store.Country = 'PL' and TransLine.quantity>0"""
   )

In [152]:
pd.read_sql(zad_d,db_conn)

,PL_Margin
0,41.63


e) What is the share of all transactions in July in stream A in individual countries that includes both category B and K inventories (at least 1 sales item each)? Only sales lines should be considered, excluding returns.

# ----------------------draft

In [13]:
pd.read_sql("""SELECT TransLine.store_no, net_amount, cost_amount, (net_amount- cost_amount) as Total 
               FROM TransLine 
               JOIN Store ON TransLine.store_no=Store.store_no""", db_conn)

,store_no,net_amount,cost_amount,Total
0,D001,49.62,23.29,26.33
1,D001,28.48,7.67,20.81
2,D001,22.17,8.22,13.95
3,D001,18.12,3.40,14.72
4,D001,70.05,34.56,35.49
...,...,...,...,...
158060,D030,91.08,57.74,33.34
158061,D030,59.32,25.80,33.52
158062,D030,11.89,7.80,4.09
158063,D030,33.15,16.91,16.24


In [26]:
dd=("""SELECT Store.Country, TransLine.store_no, TransLine.net_amount, TransLine.cost_amount, (TransLine.net_amount- TransLine.cost_amount)as Total, (TransLine.net_amount- TransLine.cost_amount)/TransLine.net_amount as Marza
       FROM TransLine
       JOIN Store
       ON TransLine.store_no=Store.store_no
       WHERE Store.Country = 'PL'
       Order by TransLine.net_amount DESC"""
   )

In [27]:
pd.read_sql(dd,db_conn)

,Country,store_no,net_amount,cost_amount,Total,Marza
0,PL,D016,400.89,232.54,168.35,0.419941
1,PL,D016,375.30,176.27,199.03,0.530322
2,PL,D016,369.71,240.15,129.56,0.350437
3,PL,D016,339.64,136.26,203.38,0.598811
4,PL,D016,338.66,296.62,42.04,0.124136
...,...,...,...,...,...,...
70226,PL,D016,-305.72,-147.88,-157.84,0.516289
70227,PL,D013,-308.15,-142.91,-165.24,0.536232
70228,PL,D016,-360.49,-169.32,-191.17,0.530306
70229,PL,D016,-372.72,-299.37,-73.35,0.196797


In [48]:
zad_a=("""SELECT Count(TransLine.transaction_no) as tc 
      FROM TransLine 
      JOIN Store
       ON TransLine.store_no=Store.store_no
       WHERE Store.Stream = 'A'
       AND TransLine.quantity>=2
       GROUP BY Store.Channel 

""")

In [49]:
pd.read_sql(zad_a,db_conn)

,tc
0,923
